In [1]:
#Read the data
import pandas as pd
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

# Drop houses where the target is missing
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

target = train_data.SalePrice

# Since missing values isn't the focus of this tutorial, we use the simplest
# possible approach, which drops these columns. 
# For more detail (and a better approach) to missing values, see
# https://www.kaggle.com/dansbecker/handling-missing-values
cols_with_missing = [col for col in train_data.columns 
                                 if train_data[col].isnull().any()]                                  
candidate_train_predictors = train_data.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)
candidate_test_predictors = test_data.drop(['Id'] + cols_with_missing, axis=1)

# "cardinality" means the number of unique values in a column.
# We use it as our only way to select categorical columns here. This is convenient, though
# a little arbitrary.
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
train_predictors = candidate_train_predictors[my_cols]
test_predictors = candidate_test_predictors[my_cols]

In [2]:
train_predictors.dtypes.sample(5)

HeatingQC       object
ScreenPorch      int64
Condition1      object
Street          object
LowQualFinSF     int64
dtype: object

In [3]:
one_hot_encoded_training_predictors = pd.get_dummies(train_predictors)

In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

predictors_without_categoricals = train_predictors.select_dtypes(exclude=['object'])

mae_without_categoricals = get_mae(predictors_without_categoricals, target)

mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, target)

print('Mean Absolute Error when Dropping Categoricals: ' + str(int(mae_without_categoricals)))
print('Mean Abslute Error with One-Hot Encoding: ' + str(int(mae_one_hot_encoded)))

Mean Absolute Error when Dropping Categoricals: 18414
Mean Abslute Error with One-Hot Encoding: 18165


In [5]:
#print(test_predictors)
test_predictors.dtypes.sample(5)

YearRemodAdd      int64
OpenPorchSF       int64
Condition2       object
SaleCondition    object
GrLivArea         int64
dtype: object

In [6]:
one_hot_encoded_test_predictors = pd.get_dummies(test_predictors)

In [7]:
final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors, join='inner', axis=1)  

In [9]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(final_train, target, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [11]:
# make predictions
predictions = my_model.predict(final_test)
print(predictions)
#from sklearn.metrics import mean_absolute_error
#print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))

[ 120861.7265625  165531.3125     171057.828125  ...,  165242.03125
  121314.0625     239459.46875  ]


In [15]:
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(final_train, target, verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [16]:
predictions2 = my_model.predict(final_test)
print(predictions2)

[ 121638.0625     170924.15625    184974.5       ...,  155400.6875
  114568.6484375  231974.484375 ]


In [17]:
my_submission = pd.DataFrame({'Id': test_data.Id, 'SalePrice': predictions})
# you could use any filename. We choose submission here
my_submission.to_csv('submission_ohexg2.csv', index=False)